## Technical Set-ups

### Command in Terminal
- conda create -n geeineq python=3.11
- conda activate geeineq
- conda install -c conda-forge mamba
- mamba install -c conda-forge pygis

#### patchly change the name of data files
`for file in 市*.geojson; do
  mv "$file" "${file/市/city-level}"
done`

### Import Packages

In [27]:
import ee
import geemap
import os
import geopandas as gpd
import pandas as pd

In [2]:
geemap.ee_initialize()

### Data Preparation

#### Step 1: Reading GEOJson Files

In [39]:
# Path to the directory containing my GEOJson files
data_directory = "./data"

# Dictionary to hold my data, with years as keys
data_by_year = {}

for file_name in os.listdir(data_directory):
    if file_name.endswith('.geojson'):
        # Extract the year from the file name 
        # (assuming the format "city-levelYEAR.geojson")
        # This slices the last 12 characters from the filename, 
        # then takes the first 4 as the year
        year = int(file_name[-12:-8])
        file_path = os.path.join(data_directory, file_name)
        print(f"Processing file: {file_path}")
        data_by_year[year] = gpd.read_file(file_path)
        print(f"Data loaded for year {year}")



Processing file: ./data/city-level1996.geojson
Data loaded for year 1996
Processing file: ./data/city-level2003.geojson
Data loaded for year 2003
Processing file: ./data/city-level2013.geojson
Data loaded for year 2013
Processing file: ./data/city-level1997.geojson
Data loaded for year 1997
Processing file: ./data/city-level2012.geojson
Data loaded for year 2012
Processing file: ./data/city-level2002.geojson
Data loaded for year 2002
Processing file: ./data/city-level2000.geojson
Data loaded for year 2000
Processing file: ./data/city-level2010.geojson
Data loaded for year 2010
Processing file: ./data/city-level1995.geojson
Data loaded for year 1995
Processing file: ./data/city-level2019.geojson
Data loaded for year 2019
Processing file: ./data/city-level2009.geojson
Data loaded for year 2009
Processing file: ./data/city-level2008.geojson
Data loaded for year 2008
Processing file: ./data/city-level2018.geojson
Data loaded for year 2018
Processing file: ./data/city-level2011.geojson
Data

##### 1. Print the first few rows of each GeoDataFrame to get a quick overview of the data:

In [40]:
# 1 Print the first few rows of each GeoDataFrame to get a quick overview of the data:
for year, data in data_by_year.items():
    print(f"Data for year {year}:")
    print(data.head())
    print("\n")

Data for year 1996:
     省     省代码      市     市代码  \
0  北京市  110000  北京市辖区  110100   
1  北京市  110000  北京市辖县  110200   
2  天津市  120000  天津市辖区  120100   
3  天津市  120000  天津市辖县  120200   
4  河北省  130000   石家庄市  130100   

                                            geometry  
0  MULTIPOLYGON (((115.95390 40.08780, 115.94240 ...  
1  MULTIPOLYGON (((116.66140 41.03630, 116.64820 ...  
2  MULTIPOLYGON (((118.05240 39.29560, 117.99580 ...  
3  MULTIPOLYGON (((117.15690 38.74750, 117.17020 ...  
4  MULTIPOLYGON (((113.84270 38.76480, 113.84310 ...  


Data for year 2003:
     省     省代码      市     市代码  \
0  北京市  110000  北京市辖区  110100   
1  北京市  110000  北京市辖县  110200   
2  天津市  120000  天津市辖区  120100   
3  天津市  120000  天津市辖县  120200   
4  河北省  130000   石家庄市  130100   

                                            geometry  
0  MULTIPOLYGON (((116.66140 41.03630, 116.64820 ...  
1  MULTIPOLYGON (((116.45630 40.77910, 116.44340 ...  
2  MULTIPOLYGON (((117.19980 39.83410, 117.17100 ...  
3  MULTIPO

##### 2. List all columns in the GeoDataFrames:

In [41]:
# 2 List all columns in the GeoDataFrames:
for year, data in data_by_year.items():
    print(f"Columns for year {year}:")
    print(data.columns)
    print("\n")

Columns for year 1996:
Index(['省', '省代码', '市', '市代码', 'geometry'], dtype='object')


Columns for year 2003:
Index(['省', '省代码', '市', '市代码', 'geometry'], dtype='object')


Columns for year 2013:
Index(['省', '省代码', '市', '市代码', 'geometry'], dtype='object')


Columns for year 1997:
Index(['省', '省代码', '市', '市代码', 'geometry'], dtype='object')


Columns for year 2012:
Index(['省', '省代码', '市', '市代码', 'geometry'], dtype='object')


Columns for year 2002:
Index(['省', '省代码', '市', '市代码', 'geometry'], dtype='object')


Columns for year 2000:
Index(['省', '省代码', '市', '市代码', 'geometry'], dtype='object')


Columns for year 2010:
Index(['省', '省代码', '市', '市代码', 'geometry'], dtype='object')


Columns for year 1995:
Index(['省', '省代码', '市', '市代码', 'geometry'], dtype='object')


Columns for year 2019:
Index(['省代码', '省', '市代码', '市', '类型', 'geometry'], dtype='object')


Columns for year 2009:
Index(['省', '省代码', '市', '市代码', 'geometry'], dtype='object')


Columns for year 2008:
Index(['省', '省代码', '市', '市代码', 'geom

##### 3. List all columns in the GeoDataFrames for year 2020 and 2021:

In [57]:
# 3 List all columns in the GeoDataFrames for year 2020 and 2021:
for year, data in data_by_year.items():
    if year in [2020, 2021]:
        print(f"Columns for year {year}:")
        print(data.columns)
        print("\n")

Columns for year 2021:
Index(['province', 'province_code', 'province_level', 'city', 'city_code',
       'city_level', 'geometry'],
      dtype='object')


Columns for year 2020:
Index(['province', 'province_code', 'province_level', 'city', 'city_code',
       'city_level', 'geometry'],
      dtype='object')




##### 4. Rename columns to English

In [43]:
# 4 Rename the columns in the GeoDataFrames for year 1990-2019 省 to province, 市 to city, 省代码 to province_code, 市代码 to city_code.
for year, data in data_by_year.items():
    if year in range(1990, 2020):
        data.rename(columns={'省': 'province', '市': 'city', '省代码': 'province_code', '市代码': 'city_code'}, inplace=True)
        print(f"Columns for year {year}:")
        print(data.columns)
        print("\n")

Columns for year 1996:
Index(['province', 'province_code', 'city', 'city_code', 'geometry'], dtype='object')


Columns for year 2003:
Index(['province', 'province_code', 'city', 'city_code', 'geometry'], dtype='object')


Columns for year 2013:
Index(['province', 'province_code', 'city', 'city_code', 'geometry'], dtype='object')


Columns for year 1997:
Index(['province', 'province_code', 'city', 'city_code', 'geometry'], dtype='object')


Columns for year 2012:
Index(['province', 'province_code', 'city', 'city_code', 'geometry'], dtype='object')


Columns for year 2002:
Index(['province', 'province_code', 'city', 'city_code', 'geometry'], dtype='object')


Columns for year 2000:
Index(['province', 'province_code', 'city', 'city_code', 'geometry'], dtype='object')


Columns for year 2010:
Index(['province', 'province_code', 'city', 'city_code', 'geometry'], dtype='object')


Columns for year 1995:
Index(['province', 'province_code', 'city', 'city_code', 'geometry'], dtype='object')


C

In [45]:
# 5 Rename the columns in the GeoDataFrames for year 2020 and 2021 since they have different column names.
for year, data in data_by_year.items():
    if year in [2020, 2021]:
        data.rename(columns={'省': 'province', '省类型': 'province_level', 
                             '市': 'city', '市类型': 'city_level', '省代码': 'province_code', 
                             '市代码': 'city_code'}, inplace=True)
        print(f"Columns for year {year}:")
        print(data.columns)
        print("\n")

Columns for year 2021:
Index(['province', 'province_code', 'province_level', 'city', 'city_code',
       'city_level', 'geometry'],
      dtype='object')


Columns for year 2020:
Index(['province', 'province_code', 'province_level', 'city', 'city_code',
       'city_level', 'geometry'],
      dtype='object')




##### 5. Generate sepreated GEOJSON files by province + year

In [58]:
# Define the years to include
years = list(range(1990, 2022))

# Get the unique provinces
provinces = set(province for gdf in data_by_year.values() for province in gdf['province'].unique())

# Loop through all provinces and years and generate the GeoJSON files
for province in provinces:
    for year in years:
        # Check if the year exists in the data
        if year in data_by_year:
            # Filter the data for the current province and year
            gdf = data_by_year[year][data_by_year[year]['province'] == province]
            
            # Add a new column 'year' to the GeoDataFrame
            gdf['year'] = year
            
            # Define the file path for the GeoJSON file
            file_path = os.path.join(data_directory, f'{province}{year}.geojson')
            
            # Export the GeoDataFrame to a GeoJSON file
            gdf.to_file(file_path, driver='GeoJSON')

In [59]:
import glob

# Define the province
province = '河北省'

# Get a list of all the GeoJSON files for the province
file_paths = glob.glob(os.path.join(data_directory, f'{province}*.geojson'))

# Initialize an empty list to store the GeoDataFrames
gdfs = []

# Loop through all the file paths and read each file
for file_path in file_paths:
    gdf = gpd.read_file(file_path)
    gdfs.append(gdf)

# Concatenate all the GeoDataFrames into a single GeoDataFrame
gdf_hebei = pd.concat(gdfs, ignore_index=True)

In [63]:
# 1 Visualize the data using geemap and visually inspect their boundaries.
# Create a Map instance
Map = geemap.Map()

# Define the years to include and their corresponding colors
years = [1990, 1991, 1992, 1994]
colors = ['red', 'green', 'blue', 'black']

# Loop through the selected years and add each GeoDataFrame as a layer
for year, color in zip(years, colors):
    # Filter the GeoDataFrame for the current year
    gdf = gdf_hebei[gdf_hebei['year'] == year]
    
    style = {
    "stroke": True,
    "color": color,  # change the color for each year
    "weight": 2,
    "opacity": 1,
    "fill": True,
    "fillColor": color,  # change the fill color for each year
    "fillOpacity": 0.1,
    }

    hover_style = {"fillOpacity": 0.6}

    Map.add_gdf(gdf, layer_name=f'City boundaries in {year}', style=style, hover_style=hover_style)

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

#### Step 2: Identifying City and City_Code Changes and Boundary Changes

In [52]:
# 1 Visualize the data using geemap and visually inspect their boundaries.
# Create a Map instance
Map = geemap.Map()

# Define the years to include and their corresponding colors
years = [1990, 1991, 1992, 1993]
colors = ['red', 'green', 'blue', 'black']

# Loop through the selected years and add each GeoDataFrame as a layer
for year, color in zip(years, colors):
    gdf = data_by_year[year]
    
    style = {
    "stroke": True,
    "color": color,  # change the color for each year
    "weight": 2,
    "opacity": 1,
    "fill": True,
    "fillColor": color,  # change the fill color for each year
    "fillOpacity": 0.1,
    }

    hover_style = {"fillOpacity": 0.6}

    Map.add_gdf(gdf, layer_name=f'City boundaries in {year}', style=style, hover_style=hover_style)

Map



Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [53]:
# Function to compare geometries between all years and 2021
def compare_geometries_to_2021():
    changes = []
    gdf2021 = data_by_year[2021]
    
    # Create a spatial index for 2021 for efficient querying
    spatial_index_2021 = gdf2021.sindex
    
    for year, gdf in data_by_year.items():
        if year == 2021:
            continue

        for _, row in gdf.iterrows():
            possible_matches_index = list(spatial_index_2021.intersection(row['geometry'].bounds))
            possible_matches = gdf2021.iloc[possible_matches_index]
            precise_matches = possible_matches[possible_matches.geometry.apply(lambda geom: geom.equals(row['geometry']))]
            
            for _, match in precise_matches.iterrows():
                # Check for name or code changes
                if row['city'] != match['city'] or row['city_code'] != match['city_code']:
                    changes.append({
                        'year': year,
                        'original_name': row['city'],
                        'new_name': match['city'],
                        'original_code': row['city_code'],
                        'new_code': match['city_code']
                    })
    
    # Convert changes to a DataFrame for easier tracking
    changes_df = pd.DataFrame(changes)
    
    # Print the DataFrame to check its contents
    print(changes_df)
    
    return changes_df

# Example usage
changes_to_2021 = compare_geometries_to_2021()

Empty DataFrame
Columns: []
Index: []
